In [37]:
from bs4 import BeautifulSoup
import urllib
import json
import requests

In [41]:
MEDIUM = "https://medium.com/"

In [46]:
def clean_json_response(response):
    return json.loads(response.text.split('])}while(1);</x>')[1])

In [3]:
url = "https://medium.com/_/api/users/af97a6a078cf/profile/stream?limit=100&source=followers"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [4]:
req = urllib.request.Request(url, headers=headers)

In [5]:
with urllib.request.urlopen(req) as response:
    the_page = response.read()

In [6]:
s = (str(the_page).split("</x>")[1])[:-1]

In [7]:
from ast import literal_eval
import re
s = literal_eval("'%s'" % s)
# re.sub(r'http\S+', "", s)

In [161]:
j =json.loads(s)

In [9]:
[(d['name'], d['postCount']) for d in j['payload']['userMeta']['authorTags']]

[('Politics', 261302),
 ('Marketing', 171362),
 ('Writing', 169430),
 ('Humor', 80623),
 ('Books', 69728),
 ('Creativity', 66016),
 ('Environment', 49420),
 ('Climate Change', 39734),
 ('College', 32550),
 ('Interview', 28916),
 ('Satire', 22657),
 ('Reading', 22609),
 ('Creative Writing', 22563),
 ('Comedy', 22205),
 ('Writing Tips', 14114),
 ('Publishing', 12506),
 ('Flash Fiction', 10560),
 ('Essay', 10290),
 ('Activism', 9644),
 ('Grief', 8670),
 ('Nonfiction', 8151),
 ('Poverty', 7199),
 ('Authors', 5971),
 ('Novel', 5902),
 ('Capitalism', 5100),
 ('Protest', 5041),
 ('Creative Process', 5024),
 ('Florida', 4513),
 ('Writing Life', 4287),
 ('Fishing', 4032),
 ('Classics', 2789),
 ('Pitching', 2596),
 ('Editing', 2478),
 ('Personal Essay', 2008),
 ('Fathers', 1950),
 ('Lawsuit', 1626),
 ('Hurricane', 1358),
 ('Readinglist', 1277),
 ('MFA', 1107),
 ('Writing Advice', 238),
 ('Hurricane Florence', 99)]

In [10]:
f = j['payload']['streamItems'][1:]

In [11]:
followers = [u['userPreview']['userId'] for u in f]
len(followers)

25

In [168]:
import urllib.parse

def parseResponse(url, args):
    params = urllib.parse.urlencode(args)
    
    if len(args) != 0:
        url = url + '?%s' % params
    
    req = urllib.request.Request(url, headers=headers)
    
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
        
    s = (str(the_page).split("</x>")[1])[:-1]
    s = literal_eval("'%s'" % s)
    
    j = json.loads(s)
    
    return j

In [213]:
def getFollowers(userId):
    print("Fetching user followers...")
    
    followers = set([])
    limit = 20
    hard_cap = 5000
    
    url = "https://medium.com/_/api/users/" + userId + "/profile/stream?limit="+str(limit)+"&source=followers"
    response = requests.get(url)
    
    j = clean_json_response(response)
    j = j['payload']
    
    total = j['references']['SocialStats'][userId]['usersFollowedByCount']
    
    count = 0
    print("total = " + str(total))
    
    while 'next' in j['paging'] and limit*count <= min(total, hard_cap):
        
        # add new users to followers set
        f = j['streamItems'][1:]
        f = [u['userPreview']['userId'] for u in f]
        for u in f: followers.add(u)
        
        # end of pagination check
        if 'to' not in j['paging']['next']: 
            break
        
        # find next page from response
        next_url = j['paging']['path']
        next_params = j['paging']['next']
        
        # parse next page
        response = requests.get(next_url, next_params)
        j = clean_json_response(response)
        j = j['payload']
        
        count += 1
    
    return list(followers)

In [236]:
def parsePostsPayload(payload, userId):
    if 'references' not in payload or 'Post' not in payload['references']: return []
    
    postIds = [k for k in payload['references']['Post']]
    
    posts = [] # post objects
    for pid in postIds:
        postObj = payload["references"]["Post"][pid]
        
        # filter out responses from claps list            
        if postObj['virtuals']['readingTime'] < 1: continue
        
        if len(postObj['virtuals']['tags']) == 0 and len(postObj['virtuals']['subtitle']) == 0:
            continue
        
        newObj = {'id': pid}
        
        newObj['clapCount'] = postObj['virtuals']['totalClapCount']
        newObj['wordCount'] = postObj['virtuals']['wordCount']
        newObj['readingTime'] = postObj['virtuals']['readingTime']
        newObj['tags'] = [tag['name'] for tag in postObj['virtuals']['tags']]
        newObj['title'] = postObj['title']
        newObj['userId'] = postObj['creatorId']
        newObj['time'] = postObj['latestPublishedAt']
        newObj['subtitle'] = postObj['virtuals']['subtitle']
        
        posts.append(newObj)
    
    return posts

In [153]:
# scrape page to get clapped posts
def getClaps(username, userId):
    print("Getting user claps...")
    
    url = MEDIUM + "@" + username + "/has-recommended?format=json"

    response = requests.get(url)
    response_dict = clean_json_response(response)
    
    payload = response_dict['payload']
    items = payload['streamItems']
    
    posts = parsePostsPayload(payload, userId) 
    
    filtered_posts = posts[:]
    for p in posts:
        if p['userId'] == userId:
            filtered_posts.remove(p)
    
    print('done')
    
    return filtered_posts

In [154]:
def getUserPosts(username, userId):

    print('Retrieving the latest posts...')

    post_ids = []
    
    url = MEDIUM + '@' + username + '/latest?format=json'
    response = requests.get(url)
    response_dict = clean_json_response(response)
    
    try:
        posts = response_dict['payload']['references']['Post']
    except:
        return []
    
    payload = response_dict['payload']
    posts = parsePostsPayload(payload, userId)
    
    print('done')
    
    return posts

In [155]:
(getClaps('RebeccaRennerFL', 'af97a6a078cf'))

Getting user claps...
done


[{'id': '4ccbfbb14314',
  'clapCount': 11,
  'wordCount': 1394,
  'readingTime': 5.960377358490566,
  'tags': ['Work', 'Work Life Balance', 'Anxiety', 'Advice'],
  'title': 'Ten Ways to Thrive in a Corporate Job if you have Anxiety',
  'userId': 'bb51e8f2c528',
  'time': 1542405250600,
  'subtitle': 'The corporate world can be intimidating, especially when you’re first starting out. Add to that a pounding heart or nagging stream of…'},
 {'id': 'e1b39c8c38ea',
  'clapCount': 1984,
  'wordCount': 1546,
  'readingTime': 5.833962264150943,
  'tags': ['Facebook', 'Social Media'],
  'title': 'The Definitive Tactical Guide to Quitting Facebook',
  'userId': 'eb358f417ed6',
  'time': 1541514661500,
  'subtitle': 'How to cover all your bases before leaving the site for good'},
 {'id': '7898ab6bc23e',
  'clapCount': 270,
  'wordCount': 2213,
  'readingTime': 9.050943396226414,
  'tags': ['Politics',
   'Elections',
   'Election 2018',
   'Donald Trump',
   'Equality'],
  'title': 'Yes, I Voted. 

In [162]:
def getUser(userId):
    url = "https://medium.com/_/api/users/" + userId + "/profile/stream?limit=25&source=followers"
    
    response = requests.get(url)
    j = clean_json_response(response)
    j = j['payload']
    
    f = j['streamItems'][1:]
    
    user = {}
    user['username'] = j['user']['username']
    user['userId'] = j['user']['userId']
    user['name'] = j['user']['name']
    user['tags'] = [tag['name'] for tag in j['userMeta']['authorTags']]
    user['followers'] = getFollowers(userId)
    user['bio'] = j['user']['bio']
    user['followerCount'] = j['references']['SocialStats'][userId]['usersFollowedByCount']
    user['followCount'] = j['references']['SocialStats'][userId]['usersFollowedCount']
    user['postCount'] = j['userMeta']['numberOfPostsPublished']
    
    userPosts = getUserPosts(user['username'], userId) # list of post objects
    user['posts'] = [p['id'] for p in userPosts] # list of pospt ids
    
    userClaps = getClaps(user['username'], userId)
    user['recommends'] = [p['id'] for p in userClaps] # list of post objects that user has clapped on
    
    return user, userPosts+userClaps

In [156]:
startId = "af97a6a078cf"
startUser, posts = getUser(startId)

Retrieving the latest posts...
done
Getting user claps...
done


In [157]:
startUser

{'username': 'RebeccaRennerFL',
 'userId': 'af97a6a078cf',
 'name': 'Rebecca Renner',
 'bio': 'Book reviewer, journalist and fiction writer. Bylines in the Atlantic, the Washington Post and lots of other places. Congrats on having an opinion!',
 'followerCount': 2277,
 'followCount': 1266,
 'postCount': 10,
 'posts': ['91d12413d41e',
  '51c708bae715',
  'd1e667e84096',
  '3abf176c0e88',
  '611e8b38ce01',
  'b8d90dcf09b6',
  'a6218acaa2ff',
  '842b69f87458',
  '44177904452b',
  'bec1dd397874'],
 'recommends': ['4ccbfbb14314', 'e1b39c8c38ea', '7898ab6bc23e']}

In [158]:
posts

[{'id': '91d12413d41e',
  'clapCount': 278,
  'wordCount': 1853,
  'readingTime': 6.992452830188679,
  'tags': ['Politics'],
  'title': 'How to Handle Everything That Could Go Wrong When You Try to Vote',
  'userId': 'af97a6a078cf',
  'time': 1541422862275,
  'subtitle': 'An election-day guide to dealing with any obstacle you may face trying to cast a ballot'},
 {'id': '51c708bae715',
  'clapCount': 15,
  'wordCount': 2261,
  'readingTime': 9.232075471698112,
  'tags': ['Writing', 'Interview', 'Nonfiction', 'Books', 'Authors'],
  'title': 'Anne Lamott on How to Hang on to Hope',
  'userId': 'af97a6a078cf',
  'time': 1541070061238,
  'subtitle': 'The author of ‘Almost Everything: Notes on Hope’ on stories that save us'},
 {'id': 'd1e667e84096',
  'clapCount': 2360,
  'wordCount': 3068,
  'readingTime': 12.777358490566037,
  'tags': ['Environment', 'Activism', 'Lawsuit', 'Florida', 'Protest'],
  'title': 'Janet Reno’s Sister Owes a Florida Developer $5 Million. She Has Zero Intention of 

In [43]:
import requests

def get_list_of_latest_posts_ids(usernames):

    print('Retrieving the latest posts...')

    post_ids = []
    
    for username in usernames:
        url = MEDIUM + '/@' + username + '/latest?format=json'
        response = requests.get(url)
        response_dict = clean_json_response(response)
        
        try:
            posts = response_dict['payload']['references']['Post']
        except:
            posts = []

        if posts:
            for key in posts.keys():
                post_ids.append(posts[key]['id'])

    return post_ids


In [38]:
posts = get_list_of_latest_posts_ids(['RebeccaRennerFL'])

Retrieving the latest posts...
{'success': True, 'payload': {'user': {'userId': 'af97a6a078cf', 'name': 'Rebecca Renner', 'username': 'RebeccaRennerFL', 'createdAt': 1469820156155, 'imageId': '1*w2A7XElItxfwl2qYv4l0XA.jpeg', 'backgroundImageId': '', 'bio': 'Book reviewer, journalist and fiction writer. Bylines in the Atlantic, the Washington Post and lots of other places. Congrats on having an opinion!', 'twitterScreenName': 'RebeccaRennerFL', 'facebookAccountId': '10153838114177168', 'allowNotes': 1, 'mediumMemberAt': 0, 'isNsfw': False, 'type': 'User'}, 'streamItems': [{'createdAt': 1543380213217, 'heading': {'text': 'Latest', 'heading': {'fallbackTitle': 'Latest', 'headingBasic': {'title': 'Latest'}, 'headingType': 'headingBasic'}}, 'randomId': 'd617ec2cef77', 'itemType': 'heading', 'type': 'StreamItem'}, {'createdAt': 1543380213218, 'postPreview': {'postId': '91d12413d41e', 'postSuggestionReasons': [{'reason': 28}]}, 'randomId': '532746eee14f', 'itemType': 'postPreview', 'type': 'S

In [41]:
def get_post_responses(posts):

    print('Retrieving the post responses...')

    responses = []

    for post in posts:
        url = MEDIUM + '/_/api/posts/' + post + '/responses'
        response = requests.get(url)
        response_dict = clean_json_response(response)
        responses += response_dict['payload']['value']

    return responses

In [42]:
get_post_responses(posts)

Retrieving the post responses...


[{'id': 'd781da384097',
  'versionId': '7ba78bc82378',
  'creatorId': '5f8317481dd4',
  'homeCollectionId': '',
  'title': 'Thank you so much for publishing this guide, Rebecca!',
  'detectedLanguage': 'en',
  'latestVersion': '7ba78bc82378',
  'latestPublishedVersion': '7ba78bc82378',
  'hasUnpublishedEdits': False,
  'latestRev': 9,
  'createdAt': 1541523723742,
  'updatedAt': 1541523752268,
  'acceptedAt': 0,
  'firstPublishedAt': 1541523752268,
  'latestPublishedAt': 1541523752268,
  'vote': False,
  'experimentalCss': '',
  'displayAuthor': '',
  'content': {'postDisplay': {'coverless': True}},
  'virtuals': {'allowNotes': True,
   'previewImage': {'imageId': '',
    'filter': '',
    'backgroundSize': '',
    'originalWidth': 0,
    'originalHeight': 0,
    'strategy': 'resample',
    'height': 0,
    'width': 0},
   'wordCount': 18,
   'imageCount': 0,
   'readingTime': 0.06792452830188679,
   'subtitle': '',
   'usersBySocialRecommends': [],
   'noIndex': False,
   'recommends'

In [44]:
def check_if_high_recommends(response, recommend_min):
    if response['virtuals']['recommends'] >= recommend_min:
        return True

In [45]:
from datetime import datetime, timedelta

def check_if_recent(response):
    limit_date = datetime.now() - timedelta(days=30)
    creation_epoch_time = response['createdAt'] / 1000
    creation_date = datetime.fromtimestamp(creation_epoch_time)

    if creation_date >= limit_date:
        return True

In [52]:
def get_user_ids_from_responses(responses, recommend_min):

    print('Retrieving user IDs from the responses...')

    user_ids = []

    for response in responses:
        recent = check_if_recent(response)
        high = check_if_high_recommends(response, recommend_min)

        if recent or high:
            user_ids.append(response['creatorId'])

    return user_ids

In [53]:
responses = get_post_responses(posts)
users = get_user_ids_from_responses(responses, 1)

Retrieving the post responses...
Retrieving user IDs from the responses...


In [54]:
users

['5f8317481dd4',
 '7fd8b4b65d69',
 '9473b7512d45',
 '39febf1564df',
 'e220a80c3389',
 'a01665798332',
 'a1cb48538dec',
 '15b0bd00ef53',
 'c991f1e2071a',
 'c78d8bead359',
 '23e9086d2c99',
 'a67e406686ca',
 '1a9a3b4e10c4',
 '467215093956',
 '9e483274bdaf',
 '820f9d04a08b',
 '590eac58b603',
 'a4c83c561046',
 'b85ff0d70903',
 '878562d90fab',
 '6eb3d7e4b327',
 'ebe106001c9',
 '1f62ed7c4bf1',
 '4ba27c97c7a2',
 'ca3a74d404a4',
 '5ab699761067',
 'fa3c2507f164',
 '3add28904024',
 '8c7afdbdf9a8']

In [ ]:
# End product:
# postId -> postObj
# userId -> userObj

# userObj -> name, username, followersId, followeesId, postsId, clapsId, categories, bio
# postObj -> author, responses, categories, title, time read, clapcount

In [226]:
def writeToFile(filename, data): # append to file
    with open(filename, mode='a', encoding='utf-8') as feedsjson:
        json.dump(data, feedsjson)

In [234]:
# takes starting user, limit
# returns -> users, post-ids (to be scraped)
def scrapeUsers(startId, limit):
    users = []
    posts = []
    
    user, p = getUser(startId)
    
    posts += p
    writeToFile('posts.json', posts)
    users.append(user)
    writeToFile('users.json', user)
    
    while len(users) < limit:
        print("iter = " + str(len(users)))
        max_f = 0
        max_u = None
        
        for uid in user['followers']:
            u, p = getUser(uid)
            users.append(u)
            writeToFile('posts.json', p)
            writeToFile('users.json', user)
            
            posts += p
        
            if u['followerCount'] > max_f:
                max_f = u['followerCount']
                max_u = u
            
            if len(users) > limit: break
            
        user = max_u
    
    return users, posts

In [237]:
users, posts = scrapeUsers('af97a6a078cf', 5000)

Fetching user followers...
total = 2277
Retrieving the latest posts...
done
Getting user claps...
done
iter = 1
Fetching user followers...
total = 43
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 68
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 152
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 9187
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 183
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 21
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 15389
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 433
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 47
Retrieving the latest posts...
Getting user claps...
done
Fetchi

Getting user claps...
done
Fetching user followers...
total = 512
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 329
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 78
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 18
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 214
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 7
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 6291
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 266
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 36
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 83
Retrieving the latest posts...
Getting user cl

total = 212
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 107
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 187
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 90
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 231
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 534
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 426
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 112
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 0
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 1477
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 223


done
Fetching user followers...
total = 386
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 5
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 83
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 229
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 0
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 91
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 101
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 506
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 278
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 179
Retrieving the latest posts...
Getting user claps...
done
Fetching user fo

Fetching user followers...
total = 185
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 1821
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 108
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 75
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 0
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 204
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 25
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 1182
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 2
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 501
Retrieving the latest posts...
Getting user claps...
done
Fetching 

ChunkedEncodingError: ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))

In [238]:
len(users)

3

In [239]:
len(posts)

19

In [251]:
with open('posts.json', 'r') as f:
    print(f.read()[-10:])

es them"}]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [252]:
writeToFile('test.json', {'garbage': 'value'})

In [253]:
writeToFile('test.json', {'garbage': 'value2'})

In [256]:
with open('test.json') as t:
    print(t.read())

{"garbage": "value"}{"garbage": "value2"}


In [257]:
users, posts = scrapeUsers('af97a6a078cf', 1)

Fetching user followers...
total = 2277
Retrieving the latest posts...
done
Getting user claps...
done


In [258]:
users

[{'username': 'RebeccaRennerFL',
  'userId': 'af97a6a078cf',
  'name': 'Rebecca Renner',
  'tags': ['Politics',
   'Marketing',
   'Writing',
   'Humor',
   'Books',
   'Creativity',
   'Environment',
   'Climate Change',
   'College',
   'Interview',
   'Satire',
   'Reading',
   'Creative Writing',
   'Comedy',
   'Writing Tips',
   'Publishing',
   'Flash Fiction',
   'Essay',
   'Activism',
   'Grief',
   'Nonfiction',
   'Poverty',
   'Authors',
   'Novel',
   'Capitalism',
   'Protest',
   'Creative Process',
   'Florida',
   'Writing Life',
   'Fishing',
   'Classics',
   'Pitching',
   'Editing',
   'Personal Essay',
   'Fathers',
   'Lawsuit',
   'Hurricane',
   'Readinglist',
   'MFA',
   'Writing Advice',
   'Hurricane Florence'],
  'followers': ['6cd7c495c494',
   '5743c2a66cd5',
   'af65b11411ae',
   '377e38b028c9',
   'f525b799a6dd',
   '4aec9d405f97',
   '1e6aadbacb78',
   'e92dd871e484',
   'f941b30155c5',
   '41a7e69fc07a',
   'ef91d328059d',
   'd7b0b2357c40',
   '95d

In [259]:
posts

[{'id': '91d12413d41e',
  'clapCount': 278,
  'wordCount': 1853,
  'readingTime': 6.992452830188679,
  'tags': ['Politics'],
  'title': 'How to Handle Everything That Could Go Wrong When You Try to Vote',
  'userId': 'af97a6a078cf',
  'time': 1541422862275,
  'subtitle': 'An election-day guide to dealing with any obstacle you may face trying to cast a ballot'},
 {'id': '51c708bae715',
  'clapCount': 15,
  'wordCount': 2261,
  'readingTime': 9.232075471698112,
  'tags': ['Writing', 'Interview', 'Nonfiction', 'Books', 'Authors'],
  'title': 'Anne Lamott on How to Hang on to Hope',
  'userId': 'af97a6a078cf',
  'time': 1541070061238,
  'subtitle': 'The author of ‘Almost Everything: Notes on Hope’ on stories that save us'},
 {'id': 'd1e667e84096',
  'clapCount': 2360,
  'wordCount': 3068,
  'readingTime': 12.777358490566037,
  'tags': ['Environment', 'Activism', 'Lawsuit', 'Florida', 'Protest'],
  'title': 'Janet Reno’s Sister Owes a Florida Developer $5 Million. She Has Zero Intention of 